# PRACTISE: Pandas Query Engine

## SETUP

In [29]:
import logging
import sys
from IPython.display import display, Markdown
import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.llms.openai import OpenAI

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
# define llm
llm = OpenAI(api_key=OPENAI_API_KEY)

In [6]:
df = pd.DataFrame(
    {
        "city": ["Toronto", "Tokyo", "Berlin"],
        "population": [2930000, 13960000, 3645000],
    }
)

df

,city,population
0,Toronto,2930000
1,Tokyo,13960000
2,Berlin,3645000


## QUERY ENGINE

In [9]:
query_engine = PandasQueryEngine(df=df, llm=llm)

response = query_engine.query("what is the city with highest population?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [13]:
response

Response(response='Tokyo', source_nodes=[], metadata={'pandas_instruction_str': "df.loc[df['population'].idxmax()]['city']", 'raw_pandas_output': 'Tokyo'})

In [16]:
response.metadata["pandas_instruction_str"]

"df.loc[df['population'].idxmax()]['city']"

## SYNTHESIZE RESPONSE

In [19]:
df

,city,population
0,Toronto,2930000
1,Tokyo,13960000
2,Berlin,3645000


In [18]:
query_engine = PandasQueryEngine(df=df, verbose=True, synthesize_response=True, llm=llm)

response = query_engine.query(
    "What is the city with the highest population? Give both the city and the population",
)

Markdown(response.response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.loc[df['population'].idxmax()]
```
> Pandas Output: city             Tokyo
population    13960000
Name: 1, dtype: object
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The city with the highest population is Tokyo, with a population of 13,960,000.

## TITANIC DATASET

In [20]:
# !wget 'https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/docs/examples/data/csv/titanic_train.csv' -O "../data/titanic_train.csv"

--2024-06-11 14:04:39--  https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/docs/examples/data/csv/titanic_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57726 (56K) [text/plain]
Saving to: ‘../data/titanic_train.csv’

../data/titanic_tra 100%[===================>]  56.37K  --.-KB/s    in 0.04s   

2024-06-11 14:04:40 (1.55 MB/s) - ‘../data/titanic_train.csv’ saved [57726/57726]



In [21]:
df = pd.read_csv("../data/titanic_train.csv")
df.shape

(891, 11)

In [24]:
query_engine = PandasQueryEngine(df=df, verbose=False, llm=llm)

In [27]:
response = query_engine.query("What is the correlation between survival and age?")

print(response.response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
-0.07722109457217755


In [31]:
display(Markdown(f"<b>{response}</b>"))

<b>-0.07722109457217755</b>

In [32]:
print(response.metadata["pandas_instruction_str"])

df['survived'].corr(df['age'])


## ADDITIONAL STEPS

In [33]:
from llama_index.core import PromptTemplate

In [34]:
query_engine = PandasQueryEngine(df=df, verbose=True)
prompts = query_engine.get_prompts()
print(prompts["pandas_prompt"].template)

You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
{df_str}

Follow these instructions:
{instruction_str}
Query: {query_str}

Expression:


In [35]:
print(prompts["response_synthesis_prompt"].template)

Given an input question, synthesize a response from the query results.
Query: {query_str}

Pandas Instructions (optional):
{pandas_instructions}

Pandas Output: {pandas_output}

Response: 


In [36]:
new_prompt = PromptTemplate(
    """
You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.

This is the result of `print(df.head())`:
{df_str}

Follow these instructions:
{instruction_str}
Query: {query_str}

Expression: """
)

query_engine.update_prompts({"pandas_prompt": new_prompt})

In [37]:
query_engine

In [38]:
instruction_str = """
1. Convert the query to executable Python code using Pandas.
2. The final line of code should be a Python expression that can be called with the `eval()` function.
3. The code should represent a solution to the query.
4. PRINT ONLY THE EXPRESSION.
5. Do not quote the expression.
"""